In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import Javascript  # Import Javascript module
import time
import pandas as pd

class StopwatchApp:
    def __init__(self):
        self.name_input = widgets.Text(placeholder="Enter Name", description="Name:")
        self.competition_input = widgets.Text(placeholder="Enter Competition", description="Competition:")
        self.date_input = widgets.DatePicker(description="Date:")
        self.timings = []
        self.current_lap_start = None
        self.timer_running = False
        self.cycle_number = 1
        self.current_phase = 0
        self.rows_generated = 0  # Track the number of rows generated

        self.output_area = widgets.Output()

        button_width = '25%'
        button_height = '40px'
        button_layout = widgets.Layout(width=button_width, height=button_height)

        button_style_1 = {
            'button_color': '#AEC6CF',
            'font_color': 'black',
            'border_color': 'black'
        }
        button_style_2 = {
            'button_color': '#FDFD96',
            'font_color': 'black',
            'border_color': 'black'
        }
        button_style_3 = {
            'button_color': '#77DD77',
            'font_color': 'black',
            'border_color': 'black'
        }
        button_style = {
            'button_color': '#cfcfc4',
            'font_color': 'black',
            'border_color': 'black'
        }

        self.start_lap_done_button = widgets.Button(description="Start/Lap/Done", layout=button_layout, style=button_style_1)
        self.delete_shot_button = widgets.Button(description="Delete Shot", layout=button_layout, style=button_style)
        self.export_button = widgets.Button(description="Export to Excel", layout=button_layout, style=button_style)
        self.append_to_sheet_button = widgets.Button(description="Append to Google Sheet", layout=button_layout, style=button_style)
        self.next_player_button = widgets.Button(description="Next Player", layout=button_layout, style=button_style)

        self.start_lap_done_button.on_click(self.start_lap_done_handler)
        self.delete_shot_button.on_click(self.delete_shot_handler)
        self.export_button.on_click(self.export_to_excel)
        self.append_to_sheet_button.on_click(self.append_to_google_sheet)
        self.next_player_button.on_click(self.next_player_handler)

        display(
            widgets.HBox([self.start_lap_done_button]),
            widgets.HBox([self.delete_shot_button, self.export_button, self.append_to_sheet_button, self.next_player_button]),
            self.name_input, self.competition_input, self.date_input, self.output_area
        )

        self.assign_space_bar_listener()

    def start_lap_done_handler(self, b):
        if not self.timer_running:
            self.current_lap_start = time.time()
            self.timer_running = True
            self.cycle_number = max([entry['Cycle'] for entry in self.timings], default=0) + 1
            self.current_phase = 0
            self.update_output(f"Started Cycle {self.cycle_number} - Preshot")
        else:
            self.lap_handler(b)

    def lap_handler(self, b):
        if self.timer_running and self.current_lap_start is not None:
            phase_names = ["Preshot", "Shotprep", "Execution", "Follow"]
            lap_name = f"{phase_names[self.current_phase]} Lap"

            lap_end = time.time()
            lap_duration = lap_end - self.current_lap_start
            self.current_lap_start = lap_end

            date_str = self.date_input.value.strftime('%Y%m%d') if self.date_input.value else 'NoDate'

            self.timings.append({
                'Cycle': self.cycle_number,
                'Lap': f"{lap_name} {self.cycle_number}",
                'Duration': lap_duration,
                'Name': self.name_input.value,
                'Competition': self.competition_input.value,
                'Date': date_str
            })
            self.update_output(f"{lap_name} {self.cycle_number}: {lap_duration:.2f} seconds")

            self.current_phase = (self.current_phase + 1) % len(phase_names)

            if self.current_phase == 0:
                self.cycle_number += 1

            self.rows_generated += 1  # Increment the row count

            if self.rows_generated % 4 == 0:  # Check if 4 rows are generated
                self.done_handler(b)

    def done_handler(self, b):
        self.timer_running = False
        self.update_output("Timer Stopped (Done)")
        self.display_recorded_values()
        self.rows_generated = 0  # Reset the row count

    def delete_shot_handler(self, b):
        if self.timings:
            max_cycle_number = max(entry['Cycle'] for entry in self.timings)
            self.timings = [entry for entry in self.timings if entry['Cycle'] != max_cycle_number]

            self.update_output(f"Deleted all shots with the latest cycle number ({max_cycle_number})")
            self.cycle_number = max_cycle_number + 1
            self.current_phase = 0  # Reset the phase to "Preshot"
            self.rows_generated = 0  # Reset the row count
            self.current_lap_start = time.time()  # Reset the timer
            self.update_output(f"Started Cycle {self.cycle_number} - Preshot")

    def export_to_excel(self, b):
        if self.timings:
            df = pd.DataFrame(self.timings)
            
            # Format the 'Date' column to 'DD/MM/YYYY'
            if 'Date' in df.columns:
                df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d').dt.strftime('%d/%m/%Y')
    
            name = self.name_input.value
            competition = self.competition_input.value
            date = self.date_input.value
    
            if date is not None:
                date_str = date.strftime('%Y%m%d')
            else:
                date_str = 'NoDate'
    
            file_name = f"{name}_{competition}_{date_str}.xlsx"
    
            try:
                df.to_excel(file_name, index=False)
                self.update_output(f"Exported to Excel: {file_name}")
                print(f"Exported to Excel: {file_name}")
            except Exception as e:
                self.update_output(f"Failed to export to Excel: {e}")
                print(f"Failed to export to Excel: {e}")

    def append_to_google_sheet(self, b):
        if self.timings:
            creds, _ = default()
            gc = gspread.authorize(creds)

            sheet_url = "https://docs.google.com/spreadsheets/d/1a4aORY_8T5veKe2FqViPi4QQzp787d6rtS6QEeQzDaU/edit#gid=0"

            sh = gc.open_by_url(sheet_url)
            worksheet = sh.get_worksheet(0)

            existing_data = pd.DataFrame(worksheet.get_all_records())

            data = pd.DataFrame(self.timings)
            combined_data = pd.concat([existing_data, data], ignore_index=True)

            worksheet.update([combined_data.columns.values.tolist()] + combined_data.values.tolist())

            self.update_output("Appended to Google Sheet")

    def next_player_handler(self, b):
        self.timings = []
        self.current_lap_start = None
        self.timer_running = False
        self.cycle_number = 1
        self.current_phase = 0
        self.rows_generated = 0  # Reset the row count
        self.update_output("Code reset for the next player")

    def display_recorded_values(self):
        if self.timings:
            df = pd.DataFrame(self.timings)
            with self.output_area:
                clear_output(wait=True)
                display(df)

    def update_output(self, message):
        with self.output_area:
            clear_output(wait=True)
            print(message)

    def assign_space_bar_listener(self):
        javascript_code = """
        document.addEventListener('keydown', function(event) {
            if (event.code === 'Space') {
                Jupyter.notebook.kernel.execute("stopwatch_app.start_lap_done_button.click()");
            }
        });
        """

        display(Javascript(javascript_code))

# Create an instance of the StopwatchApp class
stopwatch_app = StopwatchApp()


Text(value='', description='Name:', placeholder='Enter Name')

Text(value='', description='Competition:', placeholder='Enter Competition')

DatePicker(value=None, description='Date:', step=1)

Output()

<IPython.core.display.Javascript object>